# Settings

### Autoreload of ipynb

In [ ]:
%reload_ext autoreload
%autoreload 2

### Changing the path to the main directories path

In [ ]:
import os
os.chdir(r"C:\Users\amilion\Documents\GitHub\CNRL-Cortical-Column\src")
os.getcwd()

# Imports

In [3]:
from conex import *
from pymonntorch import *

from matplotlib import pyplot as plt

from L56.RefrenceFrames import RefrenceFrame
from L56.stimuli.current_base import RandomInputCurrent
from L56.synapse.vDistributor import ManualVCoder
from InputLayer.DataLoaderLayer import DataLoaderLayer
from InputLayer.synapse.LocationCoder import LocationCoder
from L423.network.SetTarget import SetTarget

from L56.tools.visualization import refrence_frame_raster
from L423.tools.visualize import show_filters

from torchvision.datasets import MNIST

# Configs

In [2]:
REFRENCE_FRAME_DIM = 23
REFRENCE_INH_DIM = 15
ITERATIONS = 240
REST_INTERVAL = 20
TEST_ITERATIONS = 100
SCREEN_SHOT_PATH = "C:\\Users\\amilion\\Desktop\\develop\\python\\NS\\records\\L5.6"

Input_Width = 28
Input_Height = 28
Crop_Window_Width = 23
Crop_Window_Height = 23
DoG_SIZE = 5

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_NUMBER = 100

OUT_CHANNEL = 8
IN_CHANNEL = 1
KERNEL_WIDTH = 13
KERNEL_HEIGHT = 13

DATASET_IMAGE_WIDTH = IMAGE_WIDTH - DoG_SIZE + 1
DATASET_IMAGE_HEIGHT = IMAGE_HEIGHT - DoG_SIZE + 1
INPUT_WIDTH = Crop_Window_Width - DoG_SIZE + 1
INPUT_HEIGHT = Crop_Window_Height - DoG_SIZE + 1

L4_WIDTH = Crop_Window_Width - KERNEL_WIDTH + 1
L4_HEIGHT = Crop_Window_Height - KERNEL_HEIGHT + 1

L23_WIDTH = L4_WIDTH//2
L23_HEIGHT = L4_HEIGHT//2

J_0 = 300
p = 0.8

# Loading Data